In [1]:
# import libraries

from bs4 import BeautifulSoup
import requests
import regex
import pandas as pd

In [2]:
movie_id = []
movie_name = []
genre = []
rating = []
overview = []
director = []
stars = []
votes = []

page_id = []

In [3]:
for i in range(0, 6000, 50):
    page_id.append(i)
    url = 'https://www.imdb.com/search/title/?title_type=feature&year=2000-01-01,2021-12-31&sort=num_votes,desc&start={}&ref_=adv_nxt'.format(
        i)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    movie_data = soup.find_all('div',
                               attrs={'class': "lister-item mode-advanced"})
    for i in movie_data:
        #       movie id
        mid = i.find('h3', 'lister-item-header')
        mid = mid.a.attrs['href']
        mid = mid.replace("/title/", '')
        mid = mid.replace("/", '')
        mid = mid.replace("?ref_=adv_li_tt", '')
        mid = mid.strip()

        # movie name
        m_name = i.find('h3', 'lister-item-header')
        m_name = m_name.a.text
        m_name = m_name.replace("</?(a|A).*?>", '')
        m_name = m_name.strip()

        #genre
        g = i.p.find('span', 'genre')
        g = str(g)
        g = g.replace('<span class="genre">', '')
        g = g.replace('</span>', '')
        g = g.strip()

        #ratings
        r = i.find('div', attrs={'class': 'ratings-bar'})
        r = i.find('strong')
        r = str(r)
        r = r.replace('<strong>', '')
        r = r.replace('</strong>', '')
        r = r.strip()

        # overview
        t = i.find_all_next('p', 'text-muted')[1]
        t = str(t)
        t = t.replace('<p class="text-muted">', '')
        t = t.replace('</p>', '')
        t = t.strip()

        # Director
        d = i.find_next('p', attrs={'class': ''})
        d = d.find('a').text
        d = str(d)
        d = d.strip()

        # actors
        a = i.find('p', attrs={'class': ''})
        a = a.find_all_next('a')[1:4]
        a = str(a)
        a = a.replace('[', '')
        a = a.replace(']', '')
        a = regex.sub(r'<a href=[\'"]?([^\'" >]+)', '', a)
        a = a.replace('"', '')
        a = a.replace('</a>', '')
        a = a.replace('>', '')
        a = a.strip()

        #votes
        v = i.find_all('span', attrs={'name': 'nv'})[0]
        v = str(v)
        v = v.split('</span>')
        v = v[0]
        v = v.split('>')
        v = v[1]

        stars.append(a)
        director.append(d)
        overview.append(t)
        rating.append(r)
        genre.append(g)
        movie_id.append(mid)
        movie_name.append(m_name)
        votes.append(v)

print(len(movie_id))

5999


In [4]:
# converting array to series

movie_id = pd.Series(movie_id)
movie_name = pd.Series(movie_name)
genre = pd.Series(genre)
director = pd.Series(director)
stars = pd.Series(stars)
overview = pd.Series(overview)
rating = pd.Series(rating)
votes = pd.Series(votes)

# concating the series
df = pd.concat(
    [movie_id, movie_name, genre, director, stars, overview, rating, votes],
    axis=1)
df.rename(columns={
    0: 'id',
    1: 'title',
    2: 'genre',
    3: 'director',
    4: 'stars',
    5: 'overview',
    6: 'rating',
    7: 'votes'
},
          inplace=True)
df = df.drop_duplicates()
df.reset_index(inplace=True, drop=True)
df['votes'] = df['votes'].str.replace(',', '')
df['votes'] = df['votes'].astype('int64')
df['rating'] = df['rating'].astype('float64')

In [5]:
df.head()

,id,title,genre,director,stars,overview,rating,votes
0,tt0468569,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart",When the menace known as the Joker wreaks havo...,9.0,2465320
1,tt1375666,Inception,"Action, Adventure, Sci-Fi",Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",A thief who steals corporate secrets through t...,8.8,2210913
2,tt0120737,The Lord of the Rings: The Fellowship of the Ring,"Action, Adventure, Drama",Peter Jackson,"Elijah Wood, Ian McKellen, Orlando Bloom",A meek Hobbit from the Shire and eight compani...,8.8,1757730
3,tt0167260,The Lord of the Rings: The Return of the King,"Action, Adventure, Drama",Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen",Gandalf and Aragorn lead the World of Men agai...,8.9,1736701
4,tt0816692,Interstellar,"Adventure, Drama, Sci-Fi",Christopher Nolan,"Matthew McConaughey, Anne Hathaway, Jessica Ch...",A team of explorers travel through a wormhole ...,8.6,1661792


In [7]:
df.to_csv('2000-2021.csv', index=False)